# 2: Working with emojis

By: Sharon Makunura

In [64]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import *
import datetime as dt
from matplotlib.ticker import MaxNLocator
import regex
import emoji
import demoji
from wordcloud import WordCloud , STOPWORDS , ImageColorGenerator
from nltk import *



In [ ]:
whatsapp_df=pd.read_csv('whatsapp_dataset.csv')

In [137]:
whatsapp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3385 entries, 0 to 3384
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    3385 non-null   datetime64[ns]
 1   Time    3385 non-null   object        
 2   Sender  3385 non-null   object        
 3   Chat    3385 non-null   object        
 4   Day     3385 non-null   object        
 5   Month   3385 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 158.8+ KB


In [117]:
#function to convert string series into list, takes series, returns list 
#null values should equal 'Empty

def stringSeries(a_series):
    stringed_s=[s for s in a_series if s!='Empty'] 
    return stringed_s


#redo above function but taking in multiple columns
#cannot work with emoji text as it reads multiple emojis as a different string
#will also return empty as a unique value

def stringCols(a_df):
    stringed_c=np.unique(a_df.values).tolist()
    return stringed_c

In [118]:
def findAllEmoji(full_emoji_df):
    emot_list=full_emoji_df.values.tolist()
    all_emot=[emoji for z in emot_list for emoji in z  if emoji!='Empty']
    return all_emot

In [120]:
#certain emojis differ by skin tone

#function seperates skin tone emojis into separate list

def st_Emojis(emoji_list):
    st='skin tone'
    st_emojis =[emot for emot in emoji_list if st in emot]
    return st_emojis

In [121]:
#write function to split by skin tone and return two lists, one with values - skin tone and other with actual skin tones

def stCheck(st_list, delimiter):
    tone='skin tone'
    nonst_list=[]
    actual_st=[]
    for each in st_list:
        split_list=each.split(delimiter)
        if split_list[0] ==each:                  #checks that there is somethin after : else will produce error
            nonst=each
        else:
            nonst = split_list[0]
            a_st=split_list[1]
            if tone in a_st:                        #caters for other emojis like flag which have non st value after :
                actual_st.append(a_st)
            else:
                nonst=nonst+':'+a_st
            
        nonst_list.append(nonst)
        
    return nonst_list, actual_st
        

In [ ]:
    #convert list into dataframe then count frequency
def list_to_df(a_list):
    
    if len(a_list)>0:
        
        words_df=pd.DataFrame(a_list)
        value_counts = words_df[0].value_counts()
        words_df=words_df.value_counts().rename_axis('unique_words').reset_index(name='counts')
    #save result of max as df;   
        max_df=words_df[words_df.counts==words_df.counts.max()]
    #some words may have same max count; combine them into one string
        if len(max_df)>1:
            words = ','.join(stringSeries(max_df.unique_words))
        else:
            words = max_df.unique_words[0]

        tot_freq = max_df.counts[0]

    else:
        words=''
        tot_freq=0
    return words, tot_freq
    


In [115]:
#create functions

#this function takes a df with message column tagged as 'Chat', and returns a df with additional columns Emoji count[number of emojis in message], Emoji text
#this function uses demoji module

def emojiCounter(a_df):
    all_eitems=0
    efinds=[]
    for row in a_df.index:
        message=a_df['Chat'][row]
        all_emojis=demoji.findall(message)
        elength=len(all_emojis)
        if elength>0:
            emoji_list=list(all_emojis.values())
            efinds.append(emoji_list)
            nfinds=len(emoji_list)
        else:
            nfinds = elength
            emoji_list=['Empty']
            
        str_emoji=', '.join(map(str,emoji_list))
        #a_df.at[row, 'Emoji count']=nfinds
        a_df.at[row, 'Emoji text']=str_emoji
        
    return a_df
            
    

In [138]:
#combine all emoji functions into one

def emojiSummary(the_df):
    the_df=the_df.iloc[:, 3:5]
    emoji_df=emojiCounter(the_df)
#splits text column so each reference to emoji is stand alone
    expanded_df=pd.concat([emoji_df, emoji_df['Emoji text'].str.split(', ', expand=True)], axis=1)
    #replace null with empty space
    expanded_df.fillna('Empty',inplace=True)
#determine unique emojis in df
#Works with the expanded columns so takes all last columns after emoji text
    x_df=expanded_df.iloc[:, 2:]
    combo_emojis=stringCols(x_df)
    listed_emojis=findAllEmoji(x_df)    #total emojis including repetitions 
    total_emojis_used=len(listed_emojis)
    
 #adjust for no emojis used

    if total_emojis_used==0:
        most_common=''
        num_used=0
        tot_unique_emojis=0
        len_st_emojis=0
        most_stemoji=''
        st_num_used=0
        most_st=''
        num_stused=0
   
        

    else:                              #emojis have been used
        
        most_common, num_used=list_to_df(listed_emojis)   #counts most common emojis(string) and how many times used

    #number of unique emojis used - 1 to account for empty 
        tot_unique_emojis=len(combo_emojis)-1   


    #determine all skin toned emojis used
        stemojis=st_Emojis(listed_emojis)
        len_st_emojis=len(stemojis)
  #adjust for no skin tone used      

        if len_st_emojis ==0:
            most_stemoji=''
            st_num_used=0
            most_st=''
            num_stused=0
        
        else:                 #skin tone used
            
            most_stemoji, st_num_used=list_to_df(stemojis)   #counts most common skin toned emojis(string) and how many times used
        #determine most popular skin tone:
            non_st,get_st=stCheck(stemojis, ':')
            most_st, num_stused=list_to_df(get_st)   #counts most common skin toned emojis(string) and how many times used

  

    
    print('Total emojis sent             :   {}'.format(total_emojis_used))
    print('Most popular emoji            :   {}'.format(most_common))
    print('Number of times used          :   {}'.format(num_used))
    print('Total unique emojis used      :   {}'.format(tot_unique_emojis))
    print('Total skin-toned emojis used  :   {}'.format(len_st_emojis))
    print('Most popular skin-toned emojis:   {}'.format(most_stemoji))
    print('Number of times used          :   {}'.format(st_num_used))
    print('Most popular skin tone        :   {}'.format(most_st))
    print('Number of times used          :   {}'.format(num_stused))


In [139]:
emojiSummary(whatsapp_df)

Total emojis sent             :   2023
Most popular emoji            :   partying face
Number of times used          :   186
Total unique emojis used      :   336
Total skin-toned emojis used  :   318
Most popular skin-toned emojis:   clapping hands: medium-dark skin tone
Number of times used          :   25
Most popular skin tone        :    medium-dark skin tone
Number of times used          :   131
